In [1]:
import pandas as pd
import numpy as np
import jieba
import jieba.posseg as pseg
import re
#import csv
import string
#from keras import models
import os
import matplotlib.pyplot as plt



In [2]:
df = pd.read_json('D:/conda_python/NLP_goods_comment/LDA_json/data_json/fund.json')     #futures  fund
df.head()
df['content'].head()

0    \n\n　　10月10日，资本邦获悉，近日银华基金管理有限公司(下称：银华基金)旗下两只基金...
1    \n\n　　9月份在全球主要股市均出现震荡的情况下，绝大多数QDII基金都遭遇了净值下跌。统...
2    \n摘要\n【博时基金王祥：国际金价或继续呈现震荡蓄势表现】国庆假日期间，国际黄金市场围绕1...
3    \n\n　　你知道你现在持有多少只基金吗？\n　　3只、5只、10只还是数不过来？\n　　前...
4    \n\n　　外部不确定暂时弱化。内部经济修复确定，进入内生增长与十四五规划政策红利预期阶段，...
Name: content, dtype: object

In [3]:
#去重复的
df.drop_duplicates(subset='content',inplace=True)  
#内容存储在content列

In [4]:
df['content'] = df['content'].str.replace("[^\u4e00-\u9fa5]", "")


<ipython-input-4-a44719e6f974>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df['content'] = df['content'].str.replace("[^\u4e00-\u9fa5]", "")


In [5]:
def length_bigger_than_10(text):
    if len(text)>=10:
        return True
    else:
        return False

#新建length列
df['length'] = df.agg({'content': length_bigger_than_10})
#根据length列来保留长度大于10的文本，并将其放入新的dataframe中
newdf = df[df['length']==True]


In [6]:
newdf.content


0      月日资本邦获悉近日银华基金管理有限公司下称银华基金旗下两只基金进入清算期分别为银华金瑞银华鑫...
1      月份在全球主要股市均出现震荡的情况下绝大多数基金都遭遇了净值下跌统计显示在剔除净值异动基金外...
2      摘要博时基金王祥国际金价或继续呈现震荡蓄势表现国庆假日期间国际黄金市场围绕美元窄幅波动但上周...
3      你知道你现在持有多少只基金吗只只只还是数不过来前两天南南的朋友阳阳说自从看了我们文章知道了不...
4      外部不确定暂时弱化内部经济修复确定进入内生增长与十四五规划政策红利预期阶段科技性价比提升可关...
                             ...                        
123    行情描述今日月日农业种植板块再度走强截至发稿农发种业登海种业涨停莫高股份亚盛集团丰乐种业北大...
124    股三大指数今日全天维持震荡整理态势最终收盘涨跌不一创业板指六连阴市场成交量大幅萎缩两市合计成...
125    行情描述今日月日农业板块逆市走高截至发稿科拓生物莫高股份涨停荃银高科隆平高科登海种业等涨超分...
126    行情描述今日月日农业种植板块走高截至发稿板块涨板块个股中荃银高科万向德农涨停苏垦农发东方集团...
127    摘要中报业绩亮眼农业板块表现活跃后市配置价值几何有哪些基金可选月日股农业板块表现活跃涨幅超猪...
Name: content, Length: 124, dtype: object

In [7]:
def stopwordslist():#加载停用词表,这个中文停用词表.txt也是要和文件放在同一目录下的喔，因为我这里用的都是相对路径
    stopwords = [line.strip() for line in open('D:/conda_python/NLP_goods_comment/stopwords-master/cn_stopwords.txt',encoding='UTF-8').readlines()]
    return stopwords

def deleteStop(sentence):     #去停用词
    stopwords=stopwordslist()
    outstr=""
    for i in sentence:
        if i not in stopwords and i!="\n":
            outstr+=i
    return outstr
def wordCut(Review):
    rec = re.sub('[%s]' % re.escape(string.punctuation), '',Review)
    fenci=jieba.lcut(rec)    #精准模式分词
    stc=deleteStop(fenci)
    seg_list=pseg.cut(stc)    #标注词性
    
    rec = [w for w,flag in seg_list if flag not in ["nr","ns","nt","nz","m","f","ul","l","r","t"] and len(w)>=2]
    document =  " ".join(rec)
    #for word,flag in seg_list:
       # if flag not in ["nr","ns","nt","nz","m","f","ul","l","r","t"]:  #去掉这些词性（人名、地名等）的词语
          #  seten.append(word)
    return document
    #Mat=[]
  #  seten = []
  #  for rec in Review:
        #seten=[]    #这里有问题
  #      rec = re.sub('[%s]' % re.escape(string.punctuation), '',rec)
   #     fenci=jieba.lcut(rec)    #精准模式分词
   #     stc=deleteStop(fenci)     #去停用词
#         sentence = list(map(lambda x: x.strip().lower() if len(x.strip().lower()) > 0 else None, jieba.cut(stc)))  # 每句话里的单词拿出来
   #     seg_list=pseg.cut(stc)    #标注词性
    #    for word,flag in seg_list:
    #        if flag not in ["nr","ns","nt","nz","m","f","ul","l","r","t"]:  #去掉这些词性（人名、地名等）的词语
     #           seten.append(word)
        
   # return seten

In [8]:
newdf['content'] = newdf.agg({'content': wordCut})

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\23058\AppData\Local\Temp\jieba.cache
Loading model cost 0.573 seconds.
Prefix dict has been built successfully.


In [9]:
corpus = newdf['content']
corpus

0      资本 获悉 基金 管理 有限公司 下称 银华 基金 旗下 基金 进入 清算 瑞银 进入 清算...
1      月份 全球 主要 股市 出现 震荡 情况 基金 遭遇 净值 下跌 统计 显示 剔除 净值 异...
2      摘要 博时基金 金价 继续 呈现 震荡 蓄势 表现 黄金市场 围绕 美元 窄幅 波动 财政 ...
3      知道 持有 基金 前两天 朋友 知道 不能 鸡蛋 放在 同一个 篮子 道理 坚持 分散 投资...
4      确定 暂时 弱化 经济 修复 确定 进入 内生 增长 规划 政策 预期 阶段 科技 性价比 ...
                             ...                        
123    行情 描述 农业 种植 板块 再度 走强 发稿 农发 种业 种业 涨停 股份 集团 丰乐 种...
124    指数 维持 震荡 整理 态势 最终 收盘 涨跌 创业板 市场 成交量 大幅 萎缩 合计 成交...
125    行情 描述 农业 板块 逆市 走高 发稿 生物 股份 涨停 种业 分析 解读 农产品 价格 ...
126    行情 描述 农业 种植 板块 走高 发稿 板块 板块 个股 万向 涨停 种业 纷纷 分析 解...
127    摘要 业绩 亮眼 农业 板块 表现 活跃 后市 配置 价值 基金 可选 农业 板块 表现 活...
Name: content, Length: 124, dtype: object

In [10]:
import pyLDAvis
import pyLDAvis.sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

vectorizer = CountVectorizer()
doc_term_matrix = vectorizer.fit_transform(corpus)
lda_model = LatentDirichletAllocation(n_components=3, random_state=888,doc_topic_prior=0.1,topic_word_prior=0.1)
lda_model.fit(doc_term_matrix)

LatentDirichletAllocation(doc_topic_prior=0.1, n_components=3, random_state=888,
                          topic_word_prior=0.1)

In [11]:
import pyLDAvis
import pyLDAvis.gensim
#import pyLDAvis.gensim_models

data = pyLDAvis.sklearn.prepare(lda_model, doc_term_matrix, vectorizer)
pyLDAvis.display(data)

D:\ProgramData\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
C:\Users\23058\AppData\Roaming\Python\Python38\site-packages\pyLDAvis\_prepare.py:228: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info  = pd.DataFrame({'saliency': saliency, 'Term': vocab, \


In [12]:
pyLDAvis.show(data) 

D:\ProgramData\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)



Note: if you're in the IPython notebook, pyLDAvis.show() is not the best command
      to use. Consider using pyLDAvis.display(), or pyLDAvis.enable_notebook().
      See more information at http://pyLDAvis.github.io/quickstart.html .

You must interrupt the kernel to end this command

Serving to http://127.0.0.1:8889/    [Ctrl-C to exit]


127.0.0.1 - - [25/Sep/2021 20:45:20] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [25/Sep/2021 20:45:20] "GET /LDAvis.css HTTP/1.1" 200 -
127.0.0.1 - - [25/Sep/2021 20:45:20] "GET /d3.js HTTP/1.1" 200 -
127.0.0.1 - - [25/Sep/2021 20:45:20] "GET /LDAvis.js HTTP/1.1" 200 -
127.0.0.1 - - [25/Sep/2021 20:45:20] code 404, message Not Found
127.0.0.1 - - [25/Sep/2021 20:45:20] "GET /favicon.ico HTTP/1.1" 404 -



stopping Server...


In [13]:
# 输出选取词特征
vocs = vectorizer.get_feature_names()
print('主题词袋：', len(vocs))
#print(vocs)
# 输出即为所有


主题词袋： 7335


D:\ProgramData\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [14]:
print(lda_model.components_)

[[ 4.77865641  0.10030833  0.1        ...  7.19559794  0.10000421
   4.74473014]
 [ 0.12331344  2.09969167  1.1        ... 14.78089355  4.43185211
   8.24254313]
 [ 3.39803015  0.1         0.1        ... 45.32350852  5.76814368
   7.31272673]]


D:\ProgramData\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [15]:
# 打印每个单词的主题的权重值
tt_matrix = lda_model.components_
id = 0
for tt_m in tt_matrix:
    tt_dict = [(name, tt) for name, tt in zip(vocs, tt_m)]
    tt_dict = sorted(tt_dict, key=lambda x: x[1], reverse=True)
    # 打印权重值大于0.6的主题词：
    # tt_dict = [tt_threshold for tt_threshold in tt_dict if tt_threshold[1] > 0.6]
    # 打印每个类别前5个主题词：
    tt_dict = tt_dict[:30]
    print('主题%d:' % (id), tt_dict)
    id+=1


主题0: [('基金', 1189.786910508635), ('市场', 869.6524994028975), ('指数', 609.9634624802569), ('投资', 465.0536968398299), ('期货', 385.52262730659305), ('黄金', 354.18157211745927), ('商品', 283.6926489500867), ('经济', 242.12913001774012), ('资金', 241.75619902751365), ('资产', 218.68402600391136), ('策略', 214.9569061168589), ('私募', 192.17708912565178), ('收益', 187.02269707769005), ('产品', 184.70472484147754), ('投资者', 181.68030880049562), ('美元', 179.3495878600915), ('铁矿石', 172.80217056452568), ('股票', 168.45988847304585), ('数据', 164.47299424507875), ('资本', 156.72595476891158), ('风险', 149.81213793470624), ('上涨', 149.7064659235362), ('股市', 139.4425651576956), ('公司', 139.40411166016693), ('全球', 138.65296784132514), ('机构', 137.70040056365144), ('管理', 137.53232414281914), ('预期', 135.8089796985011), ('表示', 131.75203651496506), ('债券', 126.42404282144206)]
主题1: [('基建', 228.61335575848602), ('基金', 214.62404934057113), ('市场', 138.77300229781957), ('行业', 114.57459743777918), ('资金', 106.54628709511938), ('投资', 104.62102

D:\ProgramData\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [16]:
def print_top_words(model, feature_names, n_top_words):
    #打印每个主题下权重较高的term
    for topic_idx, topic in enumerate(model.components_):
        print ("Topic #%d:" % topic_idx)
        print (" ".join([feature_names[i]
                        for i in topic.argsort()[:-n_top_words - 1:-1]]))
    #print
    #打印主题-词语分布矩阵
    
    #print (model.components_)   #矩阵

n_top_words=320
tf_feature_names = vectorizer.get_feature_names()
#主题词语分布矩阵
print_top_words(lda_model, tf_feature_names, n_top_words)


D:\ProgramData\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Topic #0:
基金 市场 指数 投资 期货 黄金 商品 经济 资金 资产 策略 私募 收益 产品 投资者 美元 铁矿石 股票 数据 资本 风险 上涨 股市 公司 全球 机构 管理 预期 表示 债券 表现 证券 创板 下跌 公募 估值 券商 交易 显示 涨幅 收益率 金价 业绩 出现 行情 持续 规模 配置 调整 证券报 可能 相关 创业板 国内 震荡 主要 大幅 合约 进行 来看 情况 已经 月份 认为 企业 美联储 重要 发行 平均 品种 增长 板块 份额 超过 经理 增加 走势 价格 行业 没有 发展 波动 人民币 央行 整体 影响 历史 时间 跟踪 方面 外资 分析 股指 机会 新高 继续 豆粕 推动 参与 净值 进一步 发布 记者 银行 主题 因素 成为 政策 部分 最大 主动 合作 金融 个股 买入 原油 跌幅 人士 操作 长期 迎来 进入 债市 明显 利率 关注 流动性 方式 开放 成交 持有 一季度 选择 具有 标的 创新 公布 监管 仓位 推进 联接 最新 提供 情绪 盎司 需要 反弹 正式 原油期货 相对 需求 核心 持仓 问题 期货市场 改革 连续 计划 海外 避险 权益 通胀 维持 现货 牛市 观点 判断 报道 主力 证监会 地方 非常 对冲 降低 带来 环比 趋势 比较 量化 增量 再次 短期 降息 面临 详情 压力 三大 减少 达到 稳定 了解 利好 包括 保险 注册 最高 累计 处于 今年以来 不同 积极 世纪 净流入 汇率 摘要 工作 下行 结构性 预计 山证 收盘 跨境 有色金属 盘中 证券时报 有望 工具 研究 低估 市盈率 杠杆 组合 证券日报 下降 价值 比例 突破 消息 回落 区间 互通 成立 会议 科技 外汇 布局 形成 报告 业务 召开 统计 宣布 社保 数量 不少 回调 分散 更加 水平 期货价格 成交量 总体 导读 目标 健康 二级 有效 交易日 成本 服务 看好 环境 货币 申购 套利 汽车 获得 军工 不断 支持 要闻 升温 体系 其实 作用 披露 型基金 大跌 纳入 股权 季报 上升 基础 宽松 新能源 同比 控制 保证金 账户 协议 境外 沪指 基民 热门 亏损 能够 纷纷 原因 新兴 头条 国际要闻 落地 开启 有所 国家 货币政策 流出 合计 业内人士 中证 抄底 程度 具体 保持 黑色 关系 空间 